#### Return on Assets

In [1]:
# sp500_roa_power.py

import yfinance as yf
import pandas as pd
import numpy as np
import time

# 1) Get S&P500 list from Wikipedia
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500 = pd.read_html(url)[0]
sp500.columns = [c.strip() for c in sp500.columns]
sp500 = sp500.rename(columns={"Symbol":"Ticker","Security":"Full Name"})

# 2) Helper to find the right row label
def find_label(labels, candidates):
    for cand in candidates:
        mask = labels.str.contains(cand, case=False, na=False)
        if mask.any():
            return labels[mask][0]
    return None

# 3) Function to compute ROA‐Power
def collect_roa_power(ticker, n_years=5):
    t = yf.Ticker(ticker)
    fin = t.financials          # annual income statement
    bs  = t.balance_sheet      # annual balance sheet
    if fin is None or fin.empty or bs is None or bs.empty:
        return None

    # Find net income & total assets rows
    ni_label   = find_label(fin.index.to_series(), ["net income","netearn","netprofit"])
    ta_label   = find_label(bs.index.to_series(),   ["total assets"])
    if not ni_label or not ta_label:
        return None

    ni = fin.loc[ni_label]
    ta = bs.loc[ta_label]

    # Build ROA history
    roa = []
    for date in ni.index:
        n = ni.get(date); a = ta.get(date)
        if pd.notna(n) and pd.notna(a) and a != 0:
            roa.append(n / a)
        if len(roa) >= n_years:
            break

    if len(roa) < 2:
        return None

    # Year-to-year changes & power score
    changes = np.diff(roa)
    sd       = np.std(changes, ddof=0)
    power    = float('inf') if sd == 0 else 1.0/sd

    return {
        "Ticker": ticker,
        "Full Name": t.info.get("longName", np.nan),
        "ROA History": roa,
        "ROA Changes": changes.tolist(),
        "ROA Power": power
    }

# 4) Loop & collect
results = []
for i, row in sp500.iterrows():
    rec = collect_roa_power(row["Ticker"])
    if rec:
        results.append(rec)
    time.sleep(0.5)

# 5) Build DataFrame & show
df = pd.DataFrame(results).set_index("Ticker")
print(df[["Full Name","ROA History","ROA Changes","ROA Power"]])
df.to_csv("sp500_roa_power.csv")


C:\Users\Lfran\AppData\Local\Temp\ipykernel_19428\3554925750.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return labels[mask][0]
C:\Users\Lfran\AppData\Local\Temp\ipykernel_19428\3554925750.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return labels[mask][0]
C:\Users\Lfran\AppData\Local\Temp\ipykernel_19428\3554925750.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return labels[mask][0]
C:\Users\Lfran\AppData\Lo

                             Full Name  \
Ticker                                   
MMM                         3M Company   
AOS            A. O. Smith Corporation   
ABT                Abbott Laboratories   
ABBV                       AbbVie Inc.   
ACN                      Accenture plc   
...                                ...   
XYL                         Xylem Inc.   
YUM                  Yum! Brands, Inc.   
ZBRA    Zebra Technologies Corporation   
ZBH       Zimmer Biomet Holdings, Inc.   
ZTS                        Zoetis Inc.   

                                              ROA History  \
Ticker                                                      
MMM     [0.10055683756396107, -0.16611308817714512, 0....   
AOS     [0.16469135802469137, 0.17318522667164504, 0.0...   
ABT     [0.1646154224089223, 0.07816810992433142, 0.09...   
ABBV    [0.03165114197142667, 0.036099501896652834, 0....   
ACN     [0.1298852151124028, 0.1340914450601865, 0.145...   
...                       

In [2]:
ROA = pd.read_csv("sp500_roa_power.csv", index_col=0)
print(ROA.sort_values(by="ROA", ascending=False))

KeyError: 'ROA'

### Return on Equity


In [ ]:
# sp500_roe_power.py

import yfinance as yf
import pandas as pd
import numpy as np
import time

# 1) S&P500 list
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500 = pd.read_html(url)[0]
sp500.columns = [c.strip() for c in sp500.columns]
sp500 = sp500.rename(columns={"Symbol":"Ticker","Security":"Full Name"})

# 2) Label finder
def find_label(labels, candidates):
    for cand in candidates:
        mask = labels.str.contains(cand, case=False, na=False)
        if mask.any():
            return labels[mask][0]
    return None

# 3) Compute ROE‐Power
def collect_roe_power(ticker, n_years=5):
    t = yf.Ticker(ticker)
    fin = t.financials
    bs  = t.balance_sheet
    if fin is None or fin.empty or bs is None or bs.empty:
        return None

    # Find net income & equity rows
    ni_label  = find_label(fin.index.to_series(), ["net income","netearn","netprofit"])
    eq_label  = find_label(bs.index.to_series(),   ["stockholders equity","total equity","shareholders equity"])
    if not ni_label or not eq_label:
        return None

    ni = fin.loc[ni_label]
    eq = bs.loc[eq_label]

    # Build ROE history
    roe = []
    for date in ni.index:
        n = ni.get(date); e = eq.get(date)
        if pd.notna(n) and pd.notna(e) and e != 0:
            roe.append(n / e)
        if len(roe) >= n_years:
            break

    if len(roe) < 2:
        return None

    # Changes & power
    changes = np.diff(roe)
    sd       = np.std(changes, ddof=0)
    power    = float('inf') if sd == 0 else 1.0/sd

    return {
        "Ticker": ticker,
        "Full Name": t.info.get("longName", np.nan),
        "ROE History": roe,
        "ROE Changes": changes.tolist(),
        "ROE Power": power
    }

# 4) Loop & collect
results = []
for _, row in sp500.iterrows():
    rec = collect_roe_power(row["Ticker"])
    if rec:
        results.append(rec)
    time.sleep(0.5)

# 5) Output
df = pd.DataFrame(results).set_index("Ticker")
print(df[["Full Name","ROE History","ROE Changes","ROE Power"]])
df.to_csv("sp500_roe_power.csv")


C:\Users\Lfran\AppData\Local\Temp\ipykernel_13504\325892386.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return labels[mask][0]
C:\Users\Lfran\AppData\Local\Temp\ipykernel_13504\325892386.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return labels[mask][0]
C:\Users\Lfran\AppData\Local\Temp\ipykernel_13504\325892386.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return labels[mask][0]
C:\Users\Lfran\AppData\Local

                             Full Name  \
Ticker                                   
MMM                         3M Company   
AOS            A. O. Smith Corporation   
ABT                Abbott Laboratories   
ABBV                       AbbVie Inc.   
ACN                      Accenture plc   
...                                ...   
XYL                         Xylem Inc.   
YUM                  Yum! Brands, Inc.   
ZBRA    Zebra Technologies Corporation   
ZBH       Zimmer Biomet Holdings, Inc.   
ZTS                        Zoetis Inc.   

                                              ROE History  \
Ticker                                                      
MMM     [1.0434669442998439, -1.7478676929477845, 0.27...   
AOS     [0.28330236262277675, 0.3017783561049664, 0.13...   
ABT     [0.2811765693185633, 0.14825272647203586, 0.18...   
ABBV    [1.2866165413533834, 0.4694015444015444, 0.685...   
ACN     [0.25680928666575276, 0.2674502806015326, 0.31...   
...                       

In [ ]:
ROE = pd.read_csv("sp500_roe_power.csv", index_col=0)
print(ROE.sort_values(by="ROE Power", ascending=False))

                                  Full Name  \
Ticker                                        
LNT              Alliant Energy Corporation   
ELV                   Elevance Health, Inc.   
AEE                      Ameren Corporation   
CPRT                           Copart, Inc.   
XEL                        Xcel Energy Inc.   
...                                     ...   
MSI                Motorola Solutions, Inc.   
MTCH                      Match Group, Inc.   
NCLH    Norwegian Cruise Line Holdings Ltd.   
MTD       Mettler-Toledo International Inc.   
MCK                    McKesson Corporation   

                                              ROE History  \
Ticker                                                      
LNT     [0.09851513420902341, 0.10373321528700015, 0.1...   
ELV     [0.1447416192666102, 0.1523177123085534, 0.162...   
AEE     [0.09757305596830114, 0.10150674068199841, 0.1...   
CPRT    [0.181156034992506, 0.20672290661785336, 0.235...   
XEL     [0.09917016699

### Return on Invested Capital

In [ ]:
# sp500_roic_power.py

import yfinance as yf
import pandas as pd
import numpy as np
import time

# 1) S&P500 list
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500 = pd.read_html(url)[0]
sp500.columns = [c.strip() for c in sp500.columns]
sp500 = sp500.rename(columns={"Symbol":"Ticker","Security":"Full Name"})

# 2) Label finder
def find_label(labels, candidates):
    for cand in candidates:
        mask = labels.str.contains(cand, case=False, na=False)
        if mask.any():
            return labels[mask][0]
    return None

# 3) Compute ROIC‐Power
def collect_roic_power(ticker, n_years=5):
    t = yf.Ticker(ticker)
    fin = t.financials
    bs  = t.balance_sheet
    if fin is None or fin.empty or bs is None or bs.empty:
        return None

    # Find operating income & assets & current liabilities
    op_label  = find_label(fin.index.to_series(), ["operating income","ebit","oper income"])
    ta_label  = find_label(bs.index.to_series(),   ["total assets"])
    cl_label  = find_label(bs.index.to_series(),   ["current liabilities"])
    if not op_label or not ta_label or not cl_label:
        return None

    op = fin.loc[op_label]
    ta = bs.loc[ta_label]
    cl = bs.loc[cl_label]

    # Build ROIC history (EBIT / (Assets – CL))
    roic = []
    for date in op.index:
        o = op.get(date); a = ta.get(date); c = cl.get(date)
        dc = a - c if pd.notna(a) and pd.notna(c) else None
        if pd.notna(o) and dc and dc != 0:
            roic.append(o / dc)
        if len(roic) >= n_years:
            break

    if len(roic) < 2:
        return None

    # Changes & power
    changes = np.diff(roic)
    sd       = np.std(changes, ddof=0)
    power    = float('inf') if sd == 0 else 1.0/sd

    return {
        "Ticker": ticker,
        "Full Name": t.info.get("longName", np.nan),
        "ROIC History": roic,
        "ROIC Changes": changes.tolist(),
        "ROIC Power": power
    }

# 4) Loop & collect
results = []
for _, row in sp500.iterrows():
    rec = collect_roic_power(row["Ticker"])
    if rec:
        results.append(rec)
    time.sleep(0.5)

# 5) Output
df = pd.DataFrame(results).set_index("Ticker")
print(df[["Full Name","ROIC History","ROIC Changes","ROIC Power"]])
df.to_csv("sp500_roic_power.csv")


C:\Users\Lfran\AppData\Local\Temp\ipykernel_13504\1205475924.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return labels[mask][0]
C:\Users\Lfran\AppData\Local\Temp\ipykernel_13504\1205475924.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return labels[mask][0]
C:\Users\Lfran\AppData\Local\Temp\ipykernel_13504\1205475924.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return labels[mask][0]
C:\Users\Lfran\AppData\Lo

                             Full Name  \
Ticker                                   
MMM                         3M Company   
AOS            A. O. Smith Corporation   
ABT                Abbott Laboratories   
ABBV                       AbbVie Inc.   
ACN                      Accenture plc   
...                                ...   
XYL                         Xylem Inc.   
YUM                  Yum! Brands, Inc.   
ZBRA    Zebra Technologies Corporation   
ZBH       Zimmer Biomet Holdings, Inc.   
ZTS                        Zoetis Inc.   

                                             ROIC History  \
Ticker                                                      
MMM     [0.3182838283828383, -0.5300768658566823, 0.17...   
AOS     [0.003056784262955371, 0.0024733842348639637, ...   
ABT     [0.10997776273808373, 0.1229968861547809, 0.15...   
ABBV    [0.2169638828865196, 0.2644595547079066, 0.386...   
ACN     [0.19931398008593942, 0.19811914423774904, 0.2...   
...                       